# Import data from Google Cloud Storage (GCS)

It's convenient and secure to host data in the cloud for data labeling, then sync task references to Label Studio to allow data annotators to view and label the tasks without your data leaving the secure cloud bucket. 

If your data is hosted in Google Cloud Storage (GCS), you can write a Python script to continuously sync data from the bucket with Label Studio. Follow this example to see how to do that with the [Label Studio SDK](https://labelstud.io/sdk/index.html). 

## Get your GCS bucket ready

Connect to your GCS bucket and create a list of task references that Label Studio can use, based on the contents of your bucket. 

In [ ]:
import os

BUCKET_NAME = 'your-bucket'  # specify your bucket name here
PREFIX = 'bucket/subfolder'  # specify your prefix here
# assuming your service account key is stored in GOOGLE_APPLICATION_CREDENTIALS
GOOGLE_APPLICATION_CREDENTIALS_FILE = os.getenv('GOOGLE_APPLICATION_CREDENTIALS') 
with open(GOOGLE_APPLICATION_CREDENTIALS_FILE) as f:
    credentials = f.read()

## Create a Label Studio Project

Connect to the Label Studio API with your personal API key, which you can retrieve from your user account page, and confirm you can successfully connect:

In [ ]:
from label_studio_sdk import LabelStudio
LABEL_STUDIO_URL = os.getenv("LABEL_STUDIO_URL", "http://localhost:8080")
API_KEY = os.getenv("LABEL_STUDIO_API_KEY")

ls = LabelStudio(base_url=LABEL_STUDIO_URL, api_key=API_KEY)

Create the project. In this example, the project is a basic [image object detection project](https://labelstud.io/templates/image_bbox.html):

In [ ]:
project = ls.projects.create(
    title='Using Images from GCS',
    label_config='''
    <View>
        <Image name="image" value="$image"/>
        <RectangleLabels name="objects" toName="image">
            <Label value="Airplane"/>
            <Label value="Car"/>
        </RectangleLabels>
    </View>
    '''
)

## Connect to your GCS bucket

Connect your newly-created project to your GCS bucket:

In [ ]:
storage = ls.import_storage.gcs.create(
    project=project.id,
    bucket=BUCKET_NAME,
    prefix=PREFIX,
    regex_filter='.*\.jpg',
    google_application_credentials=credentials,
    use_blob_urls=True,
    presign=True
)

## Sync tasks from GCS to Label Studio

After connecting to your bucket, you can import your private GCS links to Label Studio. When opening in Label Studio interface, they're automatically presigned for security! 

In [ ]:
ls.import_storage.gcs.sync(id=storage.id)

## Conclusion

In a few lines of code you assessed the data in your bucket, set up a new labeling project, and synced the tasks to the project. You can adapt this example to more easily create a data creation to data labeling pipeline.